<a href="https://colab.research.google.com/github/jinxuan/uberjob-demo/blob/main/uberjob_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uberjob

In [ ]:
import operator
import os
import random
import shutil
import time

import uberjob

In [ ]:
def area(width, height):
     return width * height
# before
x = 1 + 2
y = 3 + 4
z = area(x, y)
print(f'area[run by function] is {z}')

# in uberjob
plan = uberjob.Plan()
x = plan.call(operator.add, 1, 2)
y = plan.call(operator.add, 3, 4)
z = plan.call(area, width=x, height=y)
res = uberjob.run(plan, output=z)
print(f'area[run by uberjob] is {res}')

# Easy Parallism

In [ ]:
import uberjob

def dont_roll_six():
    time.sleep(random.random() * 10)
    if random.randint(1, 6) == 6:
        raise Exception('Something bad happened.')

plan = uberjob.Plan()
calls = [plan.call(dont_roll_six) for _ in range(10_000)]
uberjob.run(plan, output=calls, max_workers=32)

In [ ]:
folder_a = 'folder_a'
folder_b = 'folder_b'
os.makedirs(folder_a, exist_ok=True)
os.makedirs(folder_b, exist_ok=True)

for i in range(200): # Create 200 files to choose from in folder_a
  with open(os.path.join(folder_a, f'file_{i}.txt'), 'w') as f:
    f.write(f'This is file number {i}')

def slow_copy(src, dest):
  """Copies a file with a 1-second delay."""
  time.sleep(5)
  shutil.copy(src, dest)

if os.path.exists(folder_b):
  shutil.rmtree(folder_b)
  os.makedirs(folder_b, exist_ok=True)

In [ ]:
from uberjob.stores import PathSource
plan = uberjob.Plan()
registry = uberjob.Registry()

files_to_copy = os.listdir(folder_a)

# Create copy tasks for each selected file
copy_tasks = []
for filename in files_to_copy:
  src_path = os.path.join(folder_a, filename)
  dest_path = os.path.join(folder_b, filename)
  src_node = registry.source(plan, PathSource(src_path))
  dst_node = registry.source(plan, PathSource(dest_path, required=False))
  copy_task = plan.call(slow_copy, src_node, dest_path)
  plan.add_dependency(copy_task, dst_node)
  copy_tasks.append(copy_task)

# Run the plan to execute the copy tasks
uberjob.run(plan, registry=registry, max_workers=10) # Adjust max_workers as needed

print("Finished random files.")

In [ ]:
 uberjob.render(uberjob.run(plan, registry=registry, dry_run=True))

In [ ]:
uberjob.run(plan, registry=registry, max_workers=10) # Adjust max_workers as needed
